# Équation de Laplace 2D avec condition de Dirichlet spéciale
## u = 0 sauf en haut : u(x,1) = 1
On résout ∆u = 0 sur [0,1]² avec conditions aux bords.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

def solve_laplace_dirichlet_top(Nx, Ny=None):
    if Ny is None:
        Ny = Nx
    Lx, Ly = 1.0, 1.0
    dx, dy = Lx / (Nx - 1), Ly / (Ny - 1)
    N = Nx * Ny

    def idx(i, j):
        return i + j * Nx

    A = lil_matrix((N, N))
    b = np.zeros(N)

    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)

    for j in range(Ny):
        for i in range(Nx):
            k = idx(i, j)

            if i == 0 or i == Nx - 1 or j == 0:
                A[k, k] = 1
                b[k] = 0
            elif j == Ny - 1:
                A[k, k] = 1
                b[k] = 1
            else:
                A[k, idx(i, j)]     = -4
                A[k, idx(i+1, j)]   = 1
                A[k, idx(i-1, j)]   = 1
                A[k, idx(i, j+1)]   = 1
                A[k, idx(i, j-1)]   = 1

    u = spsolve(A.tocsr(), b)
    u_2d = u.reshape((Ny, Nx))
    return u_2d, dx, x, y

## Affichage de la solution pour N = 80

In [ ]:
N_display = 80
u_display, _, x_display, y_display = solve_laplace_dirichlet_top(N_display)
X_disp, Y_disp = np.meshgrid(x_display, y_display)

plt.figure(figsize=(6, 5))
plt.contourf(X_disp, Y_disp, u_display, levels=50, cmap='viridis')
plt.colorbar(label='Température u(x, y)')
plt.title("Solution numérique pour N = 80")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Courbe log-log de l’erreur L2 par rapport à une solution de référence

In [ ]:
N_values = [10, 20, 40, 80]
errors = []
hs = []

# Solution de référence
u_ref, h_ref, _, _ = solve_laplace_dirichlet_top(160)

def ref_downsample(u_ref, factor):
    return u_ref[::factor, ::factor]

for N in N_values:
    u_num, h, _, _ = solve_laplace_dirichlet_top(N)
    factor = 160 // N
    u_ref_interp = ref_downsample(u_ref, factor)
    error = np.linalg.norm(u_num - u_ref_interp, ord=2) * h**2
    errors.append(error)
    hs.append(h)

plt.figure()
plt.loglog(hs, errors, '-o', label='Erreur L2')
plt.xlabel("Pas d’espace h")
plt.ylabel("Erreur L2")
plt.title("Convergence en norme L2 (log-log)")
plt.grid(True, which='both')
plt.legend()
plt.show()